In [ ]:
import os
from numpy import linalg
from scipy import sparse
import math

import numpy as np
import pandas as pd

from math import sqrt, log
import statistics
import scipy.linalg
import copy

from tqdm import tqdm

In [ ]:
n_available_cores = len(os.sched_getaffinity(0))
print(n_available_cores)

14


### Данные umich rss
https://doi.org/10.15783/C7R30H


In [ ]:
data = []
with open("/content/sample_data/Z.txt") as f:
    for line in f:
        data.append([str(x) for x in line.split(sep='/n')])

In [ ]:
a = []
for i in range(len(data)):
    a.append(list([float(x) for x in data[i][0].split(sep=',')]))
d = np.matrix(np.array(a))

### Данные cu rssi
https://dx.doi.org/10.15783/C7G594


In [ ]:
file_path = '/content/sample_data/omni_16dbm (1).txt'
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append([str(x).replace(' ' , ',').replace('\n' , '') for x in line.split(sep='/n')])

In [ ]:
a = []
for i in range(len(data)):
    a.append(list([x for x in data[i][0].split(sep=',')]))

Y = np.array(a)

In [ ]:
df_ = pd.DataFrame(d)
# frame 500 location 895
df_ = df_.rename(columns={0:'x', 1:'y', 5:'frame'})
df_['x'] = np.round(df_['x'].astype(float), 4)
df_['y'] = np.round(df_['y'].astype(float), 4)
df_[4] = df_[4].astype(float)
df_['frame'] = df_['frame'].astype(float)
df_[6] = df_[6].astype(float)
df_[7] = df_[7].astype(float)
df_[8] = df_[8].astype(float)
df_[9] = df_[9].astype(float)
df_[10] = df_[10].astype(float)

df_ = df_.reset_index()

In [ ]:
df_new = pd.DataFrame({
    'x': df_['x'].repeat(5),
    'y': df_['y'].repeat(5),
    'frame' : df_['frame'].repeat(5),
    'type': df_.columns[7:].tolist() * (len(df_)),
    'value': df_.iloc[:, 7:].values.flatten()
})

In [ ]:
df = pd.pivot_table(df_new, values='value', index=['x', 'y', 'type'],
                       columns=['frame'], aggfunc='mean').reset_index().dropna().drop(columns=['x', 'y', 'type'])
print(df.shape)
Y = df.values

In [ ]:
d = np.matrix(Y)

Применим разложение по сингулярным значениям (SVD), чтобы проверить, имеет ли центрированная матрица хорошую аппроксимацию низкого ранга.

In [ ]:
row_means = np.mean(d, axis=1)
d = d - row_means

shape0 = d.shape[0]
shape1 = d.shape[1]
print(shape0, shape1)

In [ ]:
main_arr = np.array(d)
u, s, vt = np.linalg.svd(main_arr)

In [ ]:
s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2

s2matr=np.array(s2)

total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1
print(round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

32.4 % singular values to capture 90 % variance


Введём аномалии, чтобы увидеть, как это влияет на результаты.

In [ ]:

df = pd.DataFrame(d)
#используем экспоненциально-взвешенную скользящую среднюю (EWMA) для прогнозирования будущих записей на основе их прошлых значений
df_ewm = df.ewm(alpha=0.8, adjust=False).mean()
#используем максимальную разницу между фактическим и прогнозируемым значением в качестве размера вводимой аномалии
dt = df - df_ewm
max_by_columns = dt.max()
anomoly_size = max(max_by_columns)
print("Anomoly_size", anomoly_size)

#изменим долю записей для введения аномалий с 5% до 10%, а также масштабируем размер аномалии на s, который равен 0,5 или 1.
s_5 = anomoly_size*0.5
s_1 = anomoly_size*1

random_indx_y10 = np.random.randint(0, 181, size=round(182*3127*0.1))
random_indx_x10 = np.random.randint(0, 3126, size=round(182*3127*0.1))

Anomoly_size 14.753346168551886


In [ ]:
df_ = df.copy(deep=True)
for i in range(len(random_indx_y10)):
    y = int(random_indx_y10[i])
    x = int(random_indx_x10[i])
    df_.iat[y, x] =  df_.at[y, x] + s_5

df_matrix = np.asmatrix(df_)
u, s, vt = np.linalg.svd(df_matrix)

s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2

s2matr=np.array(s2)
total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1

print("With s = 0.5, fraction = 10% ", round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

With s = 0.5, fraction = 10%  64.3 % singular values to capture 90 % variance


In [ ]:
df_ = df.copy(deep=True)
for i in range(len(random_indx_y10)//2):
    y = int(random_indx_y10[i])
    x = int(random_indx_x10[i])
    df_.iat[y, x] =  df_.at[y, x] + s_5

df_matrix = np.asmatrix(df_)
u, s, vt = np.linalg.svd(df_matrix)

s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2

s2matr=np.array(s2)
total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1

print("With s = 0.5, fraction = 5% ", round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

With s = 0.5, fraction = 5%  53.300000000000004 % singular values to capture 90 % variance


In [ ]:
df_ = df.copy(deep=True)
for i in range(len(random_indx_y10)):
    y = int(random_indx_y10[i])
    x = int(random_indx_x10[i])
    df_.iat[y, x] =  df_.at[y, x] + s_1

df_matrix = np.asmatrix(df_)
u, s, vt = np.linalg.svd(df_matrix)

s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2

s2matr=np.array(s2)
total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1
print("With s = 1, fraction = 10% ", round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

With s = 1, fraction = 10%  78.60000000000001 % singular values to capture 90 % variance


In [ ]:
df_ = df.copy(deep=True)
for i in range(len(random_indx_y10)//2):
    y = int(random_indx_y10[i])
    x = int(random_indx_x10[i])
    df_.iat[y, x] =  df_.at[y, x] + s_1

df_matrix = np.asmatrix(df_)
u, s, vt = np.linalg.svd(df_matrix)

s2=[]
s2sum=0
for i in range(0, s.shape[0]):
    s2.append(s[i]**2)
    s2sum=s2sum+s[i]**2

s2matr=np.array(s2)
total_variance = 0
num_sv = 0
for i in range(0, s.shape[0]):
    if total_variance < 0.9:
        total_variance += s2matr[i]/s2sum
        num_sv += 1

print("With s = 1, fraction = 5% ", round(num_sv/s.shape[0], 3)*100, "% singular values to capture", round(total_variance*100),"% variance")

With s = 1, fraction = 5%  73.6 % singular values to capture 90 % variance


In [ ]:
num_sv

134

In [ ]:
d = np.matrix(Y)
shape0 = d.shape[0]
shape1 = d.shape[1]
print(shape0, shape1)
main_arr = np.array(d)
df = pd.DataFrame(d)

Чем вводится больше аномалий, или чем они больше по размеру, тем больший процент сингулярных чисел необходим для объяснения 90% вариативности данных.

### LENS Decomposition

In [ ]:
def soft_thresh(x, T):
    """Soft threshold function"""
    return  np.multiply(np.sign(x), np.maximum(np.abs(x) - T, 0))

In [ ]:
# new
def ADMM(D, frac_anomaly, frac_missing,m=shape0, n=shape1):

    X = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(n, n))
    Y = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(m, n))
    Z = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(m, n))
    A = np.ones((shape0, shape1))
    B = np.ones((shape0, shape0))
    C = np.eye(shape0)

    X_0 = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(n, n))
    X_1 = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(n, n))
    Y_0 = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(m, n))
    M_0 = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(n, n))
    M_1 = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(n, n))
    N = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(m, n))
    M = np.random.randint(int(np.nanmin(D)), int(np.nanmax(D)), size=(m, n))
    non_elem = np.argwhere(D != None)
    D[np.isnan(D)] = 0

    E = np.array([np.where(x == None, 1, 0) for x in D])
    W = D - np.matmul(A, X) - np.matmul(B, Y) - np.matmul(C, Z)
    for g in range(len(non_elem)):
        W[non_elem[g][0]][non_elem[g][1]] = 0
    shape_d = (m, n)    #(shape0, shape1)
    shape_X = shape_Y = shape_d
    size_d = m*n
    #nu_D - это доля записей в D, которые не являются ни пропущенными, ни ошибочными.
    nu_D = 1 - (frac_missing+frac_anomaly)/size_d

    mu = 1.01
    p = 1.05

    J = D - A@X_0 - B@Y_0 - W
    alpha = (shape_X[0]**(0.5) + shape_X[1]**(0.5))*nu_D
    beta = (2*math.log(shape_Y[0]*shape_Y[1]))**(0.5)
    gamma = 1
    theta = 10
    K = 1
    P_1 = np.eye(n)
    q1_list_1 = [1] +  [0] * (n-1)
    q1_list_2 = [1, -1] + [0] * (n-2)
    Q_1 =  scipy.linalg.toeplitz(q1_list_1, q1_list_2)
    R_1 = np.zeros(n)

    #sigma_D определяется на каждой иттерации ADM алгоритма
    #как стандартное отклонение ряда значений J[i,j], где E[i,j]=0
    tmp_nonzero = []
    indices = np.nonzero(~np.isnan(E)) # выбираем индексы, где в Е стоят 0 - то есть где не было пропуск или ошибки
    non_elem = np.column_stack(indices)
    for h in range(len(non_elem)):
        tmp_nonzero.append(J[non_elem[h][0]][non_elem[h][1]])
    sigma_D = statistics.stdev(tmp_nonzero)
    sigma = theta*sigma_D
    MAX_ITER = 400
    j = 0
    for i in tqdm(range(MAX_ITER)):
        j += 1
        #print("Iteration", i)

        #first step X
        J = 1/(K+1)*(X_1 + M_1/mu + X_0 + M_0/mu)
        t = alpha/(mu*(K+1))
        u, s, vt = scipy.linalg.svd(J, lapack_driver='gesvd')
        S = np.diag(s)
        s = soft_thresh(S, t)
        s = np.diag(s)
        X = u[:,:2].dot(np.diag(s[:2])).dot(vt[:2,:])

        #second step
        J = X - M_1/mu
        R = P_1.transpose() @ R_1 @ Q_1 + ((mu*sigma)/gamma) * J
        up, sp, vtp = np.linalg.svd(P_1.transpose() @ P_1)
        uq, sq, vtq = np.linalg.svd(Q_1.transpose() @ Q_1)
        X_1 = up @ np.divide((vtp @ R @ uq), (sp*sq.transpose() + (mu*sigma/gamma))) @ vtq

        #third step with X0
        J_0 = X-M_0/mu
        J = D- B@Y_0 - C@Z - W + M/mu
        X_0 = np.linalg.inv(A.transpose()@A + np.eye(n))@(A.transpose()@J + J_0)

        #fourth step Y
        J = Y_0 + N/mu
        t = beta/mu
        Y = soft_thresh(J, t)

        #fifth step Y_0
        J_0 = Y - N/mu
        J = D - A@X_0 - C@Z - W + M/mu
        Y_0 = np.linalg.inv(B.transpose()@B + np.eye(m))@(B.transpose()@J + J_0)

        #sixth step Z
        J = D-np.matmul(A, X_0)-np.matmul(B, Y_0)-W + M/mu
        Z = np.linalg.inv(1/(mu*sigma)*np.eye(m) + C.transpose()@C)@(C.transpose()@J)

        #seventh step W
        W = np.multiply(E, (D - A@X_0 - B@Y_0 - C@Z + M/mu))

        #eighth step sigma
        theta = 10
        J = D - A@X_0 - B@Y_0 - W
        tmp_nonzero = []
        indices = np.nonzero(~np.isnan(E))
        non_elem = np.column_stack(indices)
        for z in range(len(non_elem)):
            tmp_nonzero.append(J[non_elem[z][0]][non_elem[z][1]])

        sigma_D = statistics.stdev(tmp_nonzero)
        sigma = theta*sigma_D

        #ninth step
        #Every itteration update M, M_0, N
        M = M + mu * (D - A@X_0 - B@Y_0 - C@Z - W)
        M_0 = M_0 + mu * (X_0-X)
        M_1 = M_1 + mu*(X_1-X)
        N = N +  mu * (Y_0-Y)

        #tenth step
        #Every itteration update mu
        mu = p*mu
        #Every 100 iterations, we multiply ρ by 1.05
        if j%100 == 0:
            p_ = 1.05
            p = p*p_
        if sigma_D == 0:
            return D
        D = A @ X_0 + B @ Y_0 + C @ Z + W
    return D

In [ ]:
%%time
import copy
import random
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import TruncatedSVD
from sklearn.impute import KNNImputer
from scipy.sparse.linalg import svds
from sklearn.neighbors import KNeighborsRegressor


#ВЫБРАТЬ ТЕХНИКИ:
miss_technique = 3
anomal_technique = 2

# НАСТРОЕНЫ:
df_ = df.copy(deep=True)
D = np.asmatrix(df_)
tmp_arr = copy.deepcopy(main_arr)

new_loss_result_list = []
new_loss_result_list_2 = []
new_loss_result_list_3 = []

p_ = [0.2, 0.4, 0.8]
anomaly_size = [0.5, 1, 2, 5] #относительная амплитуда вводимых аномалий
anomaly_ratio = [0, 0.05, 0.1, 0.2, 0.4] #доля вводимых аномалий
loss_probability = [0.2, 0.4, 0.8] #доля вводимых пропусков
loss = 0.2
anomaly_r = 0.1
anomaly_s = 1


#for p in p_:
for anomaly_r in anomaly_ratio:
#for anomaly_s in anomaly_size:
# for loss in loss_probability:
    new_nmae_err_list = []
    new_nmae_err_list_2 = []
    new_nmae_err_list_3 = []
    nmae_err_list = []
    nmae_err_list_2 = []
    nmae_err_list_3 = []
    frac_missing = round(shape0*shape1*loss)
    frac_anomaly = round(shape0*shape1*anomaly_r)//2
    for i in range(10):
        if miss_technique == 1:
            random_miss_pairs = [(random.randint(0, shape0-1), random.randint(0, shape1-1)) for _ in range(round(shape0*shape1*loss))]
        elif miss_technique == 2:
            random_indx_y10_miss = np.array([i for i in range(shape0)])
            random_indx_x10_miss = np.random.randint(0, shape1-1, size=round(shape1*loss))
        elif miss_technique == 3:
            random_indx_y10_miss = np.random.randint(0, shape0-1, size=round(shape0*loss))
            random_indx_x10_miss = np.array([i for i in range(shape1)])


        list_D = []
        list_result_D = []
        list_result_D_2 = []
        list_result_D_3 = []

        tmp_arr = copy.deepcopy(main_arr)
        tmp_arr_for_svd = copy.deepcopy(tmp_arr)

        if anomal_technique == 1:
            random_anomal_pairs = [(random.randint(0, shape0), random.randint(0, shape1)) for _ in range(shape0*shape1*anomaly_r)]
            for i in range(len(random_anomal_pairs)):
                y = int(random_anomal_pairs[i][0])
                x = int(random_anomal_pairs[i][1])
                tmp_arr[y][x] += s_1
                tmp_arr_for_svd[y][x] += s_1
        else:
            random_indx_y10_anomal = np.array([i for i in range(shape0)])
            random_indx_x10_anomal = np.random.randint(0, shape1-1, size=round(shape1*anomaly_r))
            for i in range(len(random_indx_y10_anomal)):
                for j in range(len(random_indx_x10_anomal)):
                    y = int(random_indx_y10_anomal[i])
                    x = int(random_indx_x10_anomal[j])
                    tmp_arr[y][x] += s_1
                    tmp_arr_for_svd[y][x] += s_1

# for first technique
        if miss_technique == 1:
            for i in range(len(random_miss_pairs)):
                y = int(random_miss_pairs[i][0])
                x = int(random_miss_pairs[i][1])
                list_D.append(tmp_arr[y][x])
            for i in range(len(random_miss_pairs)):
                y = int(random_miss_pairs[i][0])
                x = int(random_miss_pairs[i][1])
                tmp_arr[y][x] = None
# for second, third techniques
        else:
            for i in range(len(random_indx_y10_miss)):
                for j in range(len(random_indx_x10_miss)):
                    y = int(random_indx_y10_miss[i])
                    x = int(random_indx_x10_miss[j])
                    list_D.append(tmp_arr[y][x])

            for i in range(len(random_indx_y10_miss)):
                for j in range(len(random_indx_x10_miss)):
                    y = int(random_indx_y10_miss[i])
                    x = int(random_indx_x10_miss[j])
                    tmp_arr[y][x] = None

        mean = np.nanmean(tmp_arr)
        tmp_arr_for_svd[np.isnan(tmp_arr_for_svd)] = 0
        indices = np.nonzero(np.isnan(tmp_arr_for_svd))

        U, sigma, V = np.linalg.svd(tmp_arr_for_svd)
        U = U[:, :2]
        sigma = np.diag(sigma[:2])
        V = V[:2, :]
        # Вычислим аппроксимированную матрицу
        D_approx = U @ sigma @ V

        # Применяем K ближайших соседей для локальной интерполяции:
        knn = KNeighborsRegressor(n_neighbors=2)
        matrix_interp = knn.fit(D_approx, main_arr).predict(D_approx)

        # Получаем итоговую матрицу, заменяя пропущенные значения после локальной интерполяции:
        D_svd_knn = copy.deepcopy(tmp_arr)
        D_svd_knn[np.isnan(D_svd_knn)] = matrix_interp[np.isnan(D_svd_knn)]

# для первой техники
        if miss_technique == 1:
            for i in range(len(random_miss_pairs)):
                y = int(random_miss_pairs[i][0])
                x = int(random_miss_pairs[i][1])
                list_result_D_3.append(D_svd_knn[y][x])
                list_result_D_2.append(D_approx[y][x])
# для второй и третьей техник
        else:
            for i in range(len(random_indx_y10_miss)):
                for j in range(len(random_indx_x10_miss)):
                    y = int(random_indx_y10_miss[i])
                    x = int(random_indx_x10_miss[j])
                    list_result_D_3.append(D_svd_knn[y][x])
                    list_result_D_2.append(D_approx[y][x])

        m=shape0
        n=shape1
        result_D = ADMM(tmp_arr, frac_anomaly, frac_missing, m, n)

# для первой техники
        if miss_technique == 1:
            for i in range(len(random_miss_pairs)):
                y = int(random_miss_pairs[i][0])
                x = int(random_miss_pairs[i][1])
                list_result_D.append(result_D[y][x])
# для второй и третьей техник
        else:
            for i in range(len(random_indx_y10_miss)):
                for j in range(len(random_indx_x10_miss)):
                    y = int(random_indx_y10_miss[i])
                    x = int(random_indx_x10_miss[j])
                    list_result_D.append(result_D[y][x])
        # расчет метрики ошибки NMAE
        raznost = [abs(x - y) for x, y in zip(list_D, list_result_D)]
        raznost_2 = [abs(x - y) for x, y in zip(list_D, list_result_D_2)]
        raznost_3 = [abs(x - y) for x, y in zip(list_D, list_result_D_3)]
        new_nmae_err_list.append(np.sum(raznost) / np.sum(np.abs(list_D)))
        new_nmae_err_list_2.append(np.sum(raznost_2) / np.sum(np.abs(list_D)))
        new_nmae_err_list_3.append(np.sum(raznost_3) / np.sum(np.abs(list_D)))

    new_loss_result_list.append(np.mean(new_nmae_err_list))
    new_loss_result_list_2.append(np.mean(new_nmae_err_list_2))
    new_loss_result_list_3.append(np.mean(new_nmae_err_list_3))

print(new_loss_result_list)
print(new_loss_result_list_2)
print(new_loss_result_list_3)